<a href="https://colab.research.google.com/github/NuQten/Projet_IArecognizing_bolt_nut/blob/main/MECA653_Project_English_Version.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Automized Quality Control : an AI that recognizes a screw from a nut

**Authors:  Elisa LABALLERY, Benoît DAVID, Quentin CONANEC**

# Introduction


## Why this project ?


This project can be very interesting regarding industrial applications.

In fact, it could help automate quality control by ensuring that the right components are used and assembled by an accurate identification of, in this case, bolts and nuts.

Besides, it can positivily develop inventory management : it is able to assist in keeping track of components in warehouses and reducing errors of misplacement or misidentification of tools.

Finally, with no doubt, an AI brings efficiency gains. This project can lead to faster production and execution time, along as lower costs as it may work without fatigue.

In addition, this project is obviously interesting in matter of Big Data applications. The ability to differentiate similar objects means the AI can process and learn from large datasets, improving over time and adapting to new specifications.


## State of the art


To modelize an IA working in a factory and trained to recognize bolts from nuts, we had to import a graphic database of different bolts and nuts. You will have to import this database in the files section "content" to get the code to work effectively.


## Numerical challenges


This project assesses three key points of coding : image processing, machine learning and prediction models.

First, we will have to manipulate classes and functions to segment, normalize and binarize our imported images.

Then, we are going to create a neural network model for classifying our images using transfer learning (with the VGG16 architecture).

Finally, we are going to test our model on an image database. The predictions made by the model are based on the learned parameters of the neural network. If the model was not trained properly or if it has limitations in recognizing certain features of nuts and screws, the predictions may not be accurate.


# Coding

Now, we will guide you in the process of this coding project. First, please run these little modules so that everything is settled !

In [ ]:
from skimage import morphology, transform, feature, measure, segmentation, color, io, filters
import skimage as ski
import numpy as np
import matplotlib.pyplot as plt
import os
from sklearn.cluster import KMeans
from scipy import ndimage
import random


# Image Processing

First, let's get through the image processing part.

The best way to deal with this task is to create a class.
It's main purpose is to load images, convert them to grayscale, apply gradient filters, segment them based on pixel intensity, normalize the images to a specific dimension, and finally binarize them.

This is useful, in particular, regarding the efficiency of the machine learning model we are going to create afterwards.

Commentaries complete the code for further understanding.

In [ ]:
class Image_ia():

    def __init__(self, path: str, seuil=0.025, dimension=100, label_bg=2, label_connect=2, distance_expanse=20) -> None:
        """Initializes the image to be used by the AI onwards.

        Args:
            path (str): Path to the image file.
            seuil (float, optional): Threshold between background (bg) and foreground (fg). Value ranges between [0;1]. Defaults to 0.025.
            label_bg (int, optional): Includes the background in the labeling or not. Acceptable values are {None, 1, 2}. Defaults to 2.
            label_connect (int, optional): Number of pixels between two points to connect them together. Defaults to 2.
            dimension (int, optional): Dimension of the final image. Defaults to 100.
        """
        # Image Characteristics
        self.path = path
        self.seuil = seuil
        self.dimension = dimension
        self.label_bg = label_bg
        self.label_connect = label_connect
        self.distance_expanse = distance_expanse

        # Different images
        self.img_whole = ski.io.imread(path)
        self.gray_img_whole = ski.color.rgb2gray(self.img_whole)
        self.gradient_img_whole = ski.filters.sobel(self.gray_img_whole)
        self.imgs = self.__segmentation()
        self.gray_imgs = [ski.color.rgb2gray(img) for img in self.imgs]
        self.normalize_gray_imgs = [transform.resize(img, (self.dimension, self.dimension), anti_aliasing=True) for img in self.gray_imgs]
        self.normalize_gradient_imgs = [ski.filters.sobel(img) for img in self.normalize_gray_imgs]
        self.normalize_bin_imgs = [bin_image(img) for img in self.normalize_gradient_imgs]

    def __segmentation(self):
        """Divides the main image according to the number of parts present in the photo.

        Returns:
            list : Returns a list of different images.
        """
        # Segment the image based on the gradient filter
        labels = labelisation(self.gradient_img_whole, self.seuil, self.label_bg, self.label_connect)

        # Remove labels
        labels, enum_label = remove_label(labels)

        # Expand the labels
        label_expanded = ski.segmentation.expand_labels(labels, distance=self.distance_expanse)

        # Create images of different labels
        imgs = create_img_label(label_expanded, enum_label, self.img_whole)
        for img in imgs:  # Iterate through the different images to remove the useless ones (black/white)
            if img.max == 0 or img.min == 255:
                imgs.remove(img)

        # Return the images of different labels
        return imgs


def labelisation(img, seuil=0.025, label_bg=2, label_connect=2):
    """Differentiates the different zones of each component.

    Args:
        img (array): image filtered with the Sobel filter
        seuil (float, optional): Threshold between bg and fg. Value ranges between [0;1]. Defaults to 0.025.
        label_bg (int, optional): Includes the bg in the labeling or not. Acceptable values are {None, 1, 2}. Defaults to 2.
        label_connect (int, optional): Number of pixels between two points to connect them together. Defaults to 2.

    Returns:
        array : Returns an image with different labels.
    """
    # Create an image of the same size
    markers = np.zeros_like(img)

    # Identify the foreground and background using the chosen threshold
    foreground, background = 1, 2
    markers[img < seuil] = background
    markers[img > seuil] = foreground

    # Apply segmentation
    segmentation_result = ski.segmentation.watershed(img, markers)

    # Identify the different labels
    labels = measure.label(segmentation_result == foreground, background=label_bg, connectivity=label_connect)

    # Return an image with the labels
    return labels


def remove_label(labels, pixel_min=1000):
    """Removes labels that are too small.

    Args:
        labels (array): image of different labels
        pixel_min (int, optional): minimum number of pixels required to keep a label. Defaults to 1000.
    Returns:
        (array, dict) : Returns an image with labels removed and a dictionary with their characteristics.
    """
    # Basic characteristics
    enum_label = {}  # Store the characteristics of the labels
    x, y = labels.shape  # Dimensions of the image
    get_number = True  # Check if the label exists
    n = 0  # Number of points
    is_number = 1  # Define the label to be tested

    # Loop through the labels one by one
    while get_number == True:
        list_i = []  # List of pixels in i
        list_j = []  # List of pixels in j
        n = 0       # Number of pixels

        for i in range(x):  # Loop through the image of labels
            for j in range(y):
                if labels[i][j] == is_number:  # Check if the pixel belongs to the label
                    # Add the pixel to the list
                    n += 1
                    list_i.append(i)
                    list_j.append(j)

        if n == 0:  # If 0 points detected, then there are no more labels, close the loop
            get_number = False
        elif n <= pixel_min:  # If fewer pixels than desired, then delete the label
            for i, j in zip(list_i, list_j):
                labels[i][j] = 0
        else:  # Otherwise, store the characteristics of the label
            enum_label[is_number] = {'i_min': min(list_i),
                                     'i_max': max(list_i),
                                     'j_min': min(list_j),
                                     'j_max': max(list_j),
                                     'nb_point': n
                                    }

        # Increment by 1 to test the next label
        is_number += 1

    return labels, enum_label  # Return the remaining labels and their characteristics


def create_img_label(labels, enum_label, image):
    """Creates a new image for each label and returns a list of them.

        Args:
            enum_label (dict): characteristics of each label
            label_expanded (array): image of desired labels

        Returns:
            list : Returns a list of new images.
    """
    # Create the list that will contain the future images
    new_pict = []

    # Loop for the number of labels (and therefore photos)
    for number_label, label in enum_label.items():

        dimension = max(label['i_max']-label['i_min'],  # Get the dimensions of the future image
                        label['j_max']-label['j_min'])
        segmented_image = np.zeros((dimension, dimension, 3), dtype='uint8')  # Create a black image with the dimensions

        # Replace the black image with the desired one
        for i in range(label['i_min'], label['i_max']):  # Iterate through the location of the image in the base one
            for j in range(label['j_min'], label['j_max']):
                if labels[i][j] != number_label:  # If the pixel does not belong to the label, then make it black
                    segmented_image[i - label['i_min']][j - label['j_min']] = np.array([0, 0, 0], dtype='uint8')
                else:  # Otherwise, it takes the value of the initial image
                    segmented_image[i - label['i_min']][j - label['j_min']] = image[i][j]

        # Replace the black pixels to obtain a uniform background
        for i in range(dimension):
            for j in range(dimension):
                if np.array_equal(segmented_image[i][j], np.array([0, 0, 0]), equal_nan=False):
                    if i == 0:
                        segmented_image[i][j] = segmented_image[i][j-1]
                    elif j == 0:
                        segmented_image[i][j] = segmented_image[i-1][j]
                    else:
                        segmented_image[i][j] = segmented_image[i-1][j-1]

        # Add the image to the list of images
        new_pict.append(segmented_image)

    # Return the list of images
    return new_pict


def bin_image(img, seuil=0.025):
    """Binarizes an image.

    Args:
        img (array): image to binarize
        seuil (float, optional): Binarization threshold [0;1]. Defaults to 0.025.

    Returns:
        array : Returns a binarized image.
    """
    # Create an image of the same size
    markers = np.zeros_like(img)

    # Identify the foreground and background using the chosen threshold
    foreground, background = 1, 2
    markers[img < seuil] = background
    markers[img > seuil] = foreground

    # Apply binarization
    img_watershed = ski.segmentation.watershed(img, markers)

    # Iterate through the image
    for i in range(img_watershed.shape[0]):
        for j in range(img_watershed.shape[1]):
            if img_watershed[i][j] == 2:  # If the pixel belongs to the foreground, make it black
                img_watershed[i][j] = 1.
            else:  # If the pixel belongs to the background, make it white
                img_watershed[i][j] = 0.

    # Return the binarized image
    return img_watershed


def list_files_recursively(directory):
    """Returns a list of files present in the hierarchy
    starting from the access path.
    """
    list_path = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            list_path.append(os.path.join(root, file))
    return list_path


def save_normalize_bin_img(path, dimension):
    """Saves all the photos in a dimension from
        the image files found in the hierarchy.
    """
    # List of all access paths to an image
    liste = list_files_recursively(path)

    nut = 1
    screw = 1
    os.mkdir(f'data_base_bin_{dimension}')
    os.mkdir(f'data_base_bin_{dimension}/screws')
    os.mkdir(f'data_base_bin_{dimension}/nuts')

    for path in liste:
        img = Image_ia(path, dimension=dimension)
        for bin in img.normalize_bin_imgs:
            # Save the binary image
            if "screws" in path:
                name = f'data_base_bin_{dimension}/screws/{screw}_img_bin_screw_{dimension}.png'
                ski.io.imsave(name, (bin * 255).astype('uint8'))
                screw += 1
            else:
                name = f'data_base_bin_{dimension}/nuts/{nut}_img_bin_nut_{dimension}.png'
                ski.io.imsave(name, (bin * 255).astype('uint8'))
                nut += 1
            print(name)


if __name__ == "__main__":

    path = 'boulon_entier.jpg'  # Access path of the image
    img = Image_ia(path)

    # Display all states of the image
    plt.figure()
    plt.title('whole image (color)')
    plt.imshow(img.img_whole)
    plt.show()

    plt.figure()
    plt.title('whole image (gray)')
    plt.imshow(img.gray_img_whole, cmap='gray')
    plt.show()

    plt.figure()
    plt.title('whole image (gradient)')
    plt.imshow(img.gradient_img_whole, cmap='gray')
    plt.show()

    plt.figure()
    plt.title('segmented image (color)')
    plt.imshow(img.imgs[0])
    plt.show()

    plt.figure()
    plt.title('segmented image (gray)')
    plt.imshow(img.gray_imgs[0], cmap='gray')
    plt.show()

    plt.figure()
    plt.title('segmented normalized image (gray)')
    plt.imshow(img.normalize_gray_imgs[0], cmap='gray')
    plt.show()

    plt.figure()
    plt.title('segmented normalized image (gradient)')
    plt.imshow(img.normalize_gradient_imgs[0], cmap='gray')
    plt.show()

    plt.figure()
    plt.title('segmented normalized image (binary)')
    plt.imshow(img.normalize_bin_imgs[0], cmap='gray')
    plt.show()

    # Save all images in a dimension
    path = 'data_base'  # Access path of the images
    save_normalize_bin_img(path, 224)  # Save the images in a folder


# Machine learning

Now, we have to create an AI that manages to recognizes a screw from a nut and automatically saves the image in a folder regarding wether it's one or another.

This code defines, trains, and evaluates a neural network model for classifying our images using transfer learning with the VGG16 architecture.

To be precise, a neural network is very useful in machine learning as it is effective for image classification tasks. Besides, the VGG16 architecure contains 16 layers and the last layers are removed, so that new layers are added for classifying nuts and screws specifically, leading to an ability to extract useful feature whilst adapting it to our specific case.

Please find useful commentary in the code for further understanding.
P.S : This module may take quite a few minutes to be fully ran, please be patient !

In [ ]:
import tensorflow as tf

CLASS_NAMES = ['nut', 'screw'] #List of possibilities
dimension = 224 #Image dimension
batch_size = 32
path = f'data_base_color_{dimension}' #Path to the database

#Load training images
train_ds = tf.keras.utils.image_dataset_from_directory(
  path,
  validation_split = 0.2,
  subset = "training",
  seed = 123,
  color_mode = 'rgb',
  image_size = (dimension, dimension),
  batch_size = batch_size
)

#Load test images
test_ds = tf.keras.utils.image_dataset_from_directory(
  path,
  validation_split = 0.2,
  subset = "validation",
  seed = 123,
  color_mode = 'rgb', #'grayscale'
  image_size = (dimension, dimension),
  batch_size = batch_size
)

#Allows data to be extracted more easily and thus avoids blocking
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

#Create the model
model = tf.keras.Sequential([ #Add the model layers
    tf.keras.applications.VGG16(weights='imagenet', include_top=False, input_shape=(dimension, dimension, 3)),
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(2) #2 outputs for the 2 possibilities
])

#Compile the model
model.compile(
    optimizer = 'adam', #Model optimizer
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),   #Measures model accuracy
    metrics = ['accuracy']   #Used to monitor training and test steps
)

#Display a summary of the model
model.summary()


#Train the model
history = model.fit(
            train_ds, #Training data
            validation_data = val_ds, #Test data
            epochs = 15 #Number of epochs
)

#Save the model
model.save('model_reconize_screw_nut.h5')

# Model predictions

Here, we decided to test out our code on an image database we previously built. This code loads the pre-trained model (that was previously trained in the machine learning part), and then uses the loaded model to make predictions on a test image of either a nut or a screw.

In [2]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

CLASS_NAMES = ['nut', 'screw']

if __name__ == '__main__':
    # Load the previously trained model
    model = tf.keras.models.load_model('model_reconize_screw_nut.h5')

    # Image path
    img_path = '0_data_base_bin_nut.png'
    # Load the image to be tested
    img = tf.keras.preprocessing.image.load_img(img_path, target_size=(224, 224))
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = tf.keras.applications.vgg16.preprocess_input(img_array)

    # Model predictions on the image
    probability_model = tf.keras.Sequential([model,
                                            tf.keras.layers.Softmax()]) # Output as probability
    predictions = probability_model.predict(img_array) # Make a prediction on img_test

    # Display the AI prediction
    print(CLASS_NAMES[np.argmax(predictions[0])])

1/1 [==============================] - 1s 703ms/step
screw


# Results

Overall, these codes form a complete pipeline for image classification, starting from image preprocessing and model training to making predictions on new images. The result will be the predicted class (either 'nut' or 'screw') for each input image, giving a trusted result to the user.
